https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html?highlight=group

https://stackoverflow.com/questions/36392735/how-to-combine-multiple-rows-into-a-single-row-with-pandas

In [2]:
import numpy as np
import pandas as pd
pd.set_option('mode.chained_assignment', None)
import freeman

1. Carregando os datasets

In [4]:
atbats_csv = pd.read_csv('atbats.csv')
games_csv = pd.read_csv('games.csv')
player_names_csv = pd.read_csv('player_names.csv')

In [55]:
player_names_csv[player_names_csv['id'] == 623352]

,id,first_name,last_name
1025,623352,Josh,Hader


2. Combinacao dos datasets de jogos e rebatedores em relacao ao identificador do jogo

In [5]:
merged = pd.merge(games_csv, atbats_csv, how="inner", on=["g_id"])
games = merged.groupby(by=['g_id'])

3. Criando dicionario para facilitar o manejamento de informacao por jogo disputado.

In [6]:
games_teams = {}
for gameId in games.groups:
    game = games.get_group(gameId)
    games_teams[gameId] = {
        "home_team": {
            "name": game.home_team.values[0],
            "batters": [],
            "pitchers": []
        },
        "away_team": {
            "name": game.away_team.values[0],
            "batters": [],
            "pitchers": []
        }
    }

    batters = game.groupby(by="batter_id")
    for batterId in batters.groups:
        batter = batters.get_group(batterId)
        if batter.top.max() != True:
            games_teams[gameId]["home_team"]["batters"].append(batterId)
        else:
            games_teams[gameId]["away_team"]["batters"].append(batterId)

    pitchers = game.groupby(by="pitcher_id")
    for pitcherId in pitchers.groups:
        pitcher = pitchers.get_group(pitcherId)
        if pitcher.top.max() == True:
            games_teams[gameId]["home_team"]["pitchers"].append(pitcherId)
        else:
            games_teams[gameId]["away_team"]["pitchers"].append(pitcherId)

In [7]:
game_data = {}
for gameId in games.groups:
    game = games.get_group(gameId)
    game_data[gameId] = {
        game.home_team.values[0]: [],
        game.away_team.values[0]: []
    }

    batters = game.groupby(by="batter_id")
    for batterId in batters.groups:
        batter = batters.get_group(batterId)
        if batter.top.max() != True:
            game_data[gameId][game.home_team.values[0]].append(batterId)
        else:
            game_data[gameId][game.away_team.values[0]].append(batterId)

    pitchers = game.groupby(by="pitcher_id")
    for pitcherId in pitchers.groups:
        pitcher = pitchers.get_group(pitcherId)
        if pitcher.top.max() == True:
            game_data[gameId][game.home_team.values[0]].append(pitcherId)
        else:
            game_data[gameId][game.away_team.values[0]].append(pitcherId)

In [8]:
player_ids = player_names_csv["id"].values #65

In [9]:

for game, teams in game_data.items():
    for team, players in teams.items():
        game_data[game][team] = list(set(players))
connections = {}
for game in game_data.values():
    for team, players in game.items():
        for index in range(len(players) - 2):
            player = players[index]
            player_connections = connections.get(player, None)
            if not player_connections:
                connections[player] = set()
            for i in range(index + 1, len(players)):
                partner = players[i]
                connection = connections.get(partner, None)
                if not connection or player not in connection:
                    connections[player].add(partner)
for player in player_ids:
    if player not in connections:
        connections[player] = []

        

4. Criacao da rede no arquivo .gml

In [10]:
gml = 'graph [\n\tdirected 0\n'
for player in connections.keys():
    gml += f"\n\tnode [\n\t\tid {player}\n\t]"
gml += '\n'
for player, player_connections in connections.items():
    for connection in player_connections:
        gml += f'\n\tedge [\n\t\tsource {player}\n\t\ttarget {connection}\n\t]'
gml += '\n]'
with open('network.gml', 'w') as f:
    f.write(gml)

5. Cálculo das estatísticas dos Pitchers

Extraindo dados de WHIP

In [11]:
pitcher_data = {}
for index, row in atbats_csv.iterrows():
    pitcher_id = row['pitcher_id']
    pitcher = pitcher_data.get(pitcher_id, None)
    if not pitcher:
        pitcher_data[pitcher_id] = {
            'walks': 0,
            'hits': 0,
            'at-bats': 0,
            'IP': {
                '2015': 0,
                '2016': 0,
                '2017': 0,
                '2018': 0
            },
            'ERA': {
                '2015': 0,
                '2016': 0,
                '2017': 0,
                '2018': 0
            }
        }
    pitcher_data[pitcher_id]['at-bats'] += 1
    year = str(row['ab_id'])[:4]
    at_bat_result = row['event']
    if at_bat_result in ['Walk', 'Intent Walk', 'Hit By Pitch']:
        pitcher_data[pitcher_id]['walks'] += 1
    elif at_bat_result in ['Double', 'Single', 'Triple', 'Home Run']:
        pitcher_data[pitcher_id]['hits'] += 1
    elif at_bat_result in ['Groundout', 'Strikeout', 'Runner Out', 'Flyout', 'Forceout', 'Pop out', 'Lineout', 'Sac Bunt', 'Bunt Groundout', 'Sac Fly', 'Fielders Choice Out', 'Bunt Pop Out', 'Bunt Lineout']:
        pitcher_data[pitcher_id]['IP'][year] += 1/3
    elif at_bat_result in ['Double Play', 'Grounded Into DP', 'Strikeout - DP', 'Sac Fly DP', 'Sacrifice Bunt DP']:
        pitcher_data[pitcher_id]['IP'][year] += 2/3
    elif at_bat_result == 'Triple Play':
        pitcher_data[pitcher_id]['IP'][year] += 1
    

In [12]:
for pitcher in pitcher_data.keys():
    try:
        pitcher_data[pitcher]['WHIP'] = (pitcher_data[pitcher]['walks'] + pitcher_data[pitcher]['hits'])/sum([ip for ip in pitcher_data[pitcher]['IP'].values()])
    except:
        pitcher_data[pitcher]['WHIP'] = None

Extraindo ERA

In [65]:
for i in range(2015, 2019):
    stats = pd.read_csv(f'player_stats_{i}.csv')
    for pitcher in pitcher_data.keys():
        try:
            if pitcher_data[pitcher]['IP'][str(i)] > 0:
                pitcher_data[pitcher]['ERA'][str(i)] = stats.loc[stats.player_id == pitcher].era.values[0]
            else:
                pitcher_data[pitcher]['ERA'][str(i)] = 0
        except:
            pitcher_data[pitcher]['ERA'][str(i)] = 0


595235 2016
543964 2016
592426 2016
516414 2016
572021 2016
605304 2016
605218 2016
516714 2016
543901 2016
596331 2016
518715 2016
572362 2016
543391 2016
595465 2016
457456 2016
592127 2016
449104 2016
458537 2016
596112 2016
621385 2016
501936 2016
543532 2016
519437 2016
543017 2016
657670 2016
595345 2016
502457 2016
571035 2016
643325 2016
643297 2016
570714 2016
572990 2016
456124 2016
457779 2016
657205 2016
605525 2016
572033 2016
643550 2016
599683 2016
518883 2016
643338 2016
571969 2016
542884 2016
594027 2016
621219 2016
517448 2016
514913 2016
545357 2016
500765 2016
642229 2016
571963 2016
573188 2016
572208 2016
623439 2016
542953 2016
546276 2016
542947 2016
608717 2016
607320 2016
542194 2016
606930 2016
502285 2016
543776 2016
608638 2016
641501 2016
539438 2016
573589 2016
461865 2016
592716 2016
489197 2016
615868 2016
519344 2016
591693 2016
460077 2016
592614 2016
606291 2016
608337 2016
592779 2016
572365 2016
446381 2016
600301 2016
534606 2016
596049 2016
5924

In [70]:
for pitcher in pitcher_data.keys():
        try:
                pitcher_data[pitcher]['cERA'] = sum([ip*era for ip, era in zip(pitcher_data[pitcher]['IP'].values(), pitcher_data[pitcher]['ERA'].values())])/sum([ip for ip in pitcher_data[pitcher]['IP'].values()])
        except:
               pitcher_data[pitcher]['cERA'] = None

In [71]:
data = pd.DataFrame({
    'pitcher_id': [n for n in pitcher_data.keys()],
    'walks': [n['walks'] for n in pitcher_data.values()],
    'hits': [n['hits'] for n in pitcher_data.values()],
    'at-bats': [n['at-bats'] for n in pitcher_data.values()],
    'IP': [sum([ip for ip in n['IP'].values()]) for n in pitcher_data.values()],
    'WHIP': [n['WHIP'] for n in pitcher_data.values()],
    'cERA': [n['cERA'] for n in pitcher_data.values()]
})

In [72]:
data.to_csv('pitcher_data.csv') #138